In [ ]:
# Imports nécessaires

%matplotlib inline

# Suppression de l'affichage des messages d'avertissement
import warnings
warnings.filterwarnings('ignore')
import string
import time
import nltk
import tensorflow as tf
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import spacy
import pandas as pd
import nltk
import keras
from keras import layers
from sparknlp.base import *

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn import metrics
from sklearn import model_selection
from sklearn import set_config
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.dummy import DummyClassifier
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, f1_score, accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.multiclass import OneVsRestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import FunctionTransformer, MinMaxScaler
from sklearn.pipeline import Pipeline, FeatureUnion, make_pipeline
from sklearn.tree import DecisionTreeClassifier
from nltk.stem.snowball import FrenchStemmer
from keras.layers import TextVectorization
from keras.layers import Embedding
from spacy.lang.fr import French 


# Pour éviter l'affichage tronqué des descriptions
pd.set_option('display.max_colwidth', -1) 
# Pour la visualisation des pipelines sklearn
set_config(display='diagram') 

Définition des différentes fonctions utilisé plus tard:
get_vectorizer permet la vectorisation du texte et
load_embenddings fait ce que son nom indique

In [ ]:
def get_vectorizer(documents, max_voc_size=8000, max_seq_length=50, batch_size=128):
  vectorizer = TextVectorization(max_tokens=max_voc_size, 
                                 output_sequence_length=max_seq_length)
  # Création du jeu de données à partir de X_train et constitution de lots de 128 instances
  text_ds = tf.data.Dataset.from_tensor_slices(documents).batch(batch_size)
  # Création du vocabulaire à partir des données d'entrée
  vectorizer.adapt(text_ds)
  return vectorizer

def load_embeddings(embeddings_file):
  embeddings_index = {}
  with open(embeddings_file, 'r', encoding='utf8') as f:
      try:
        for line in f:
            word, coefs = line.split(maxsplit=1)
            coefs = np.fromstring(coefs, "f", sep=" ")
            embeddings_index[word] = coefs
      except:
        ; #un seul mot qui n'est pas lisible en utf8 donc il est ignoré
  print(f'{len(embeddings_index)} vecteurs de mots ont été lus')
  return embeddings_index


In [ ]:
film_df = pd.read_csv("allocine_genres_train.csv", sep=",")
# Liste des classes
class_names = sorted(film_df.genre.unique())
# On associe à chaque classe un identifiant unique
class_index = {class_names[i]:i for i in range(len(class_names))}

Pour le modèle (model.txt) utilisé pour le plongement de mots, il faut prendre sur le site http://vectors.nlpl.eu/repository/# le modèle ayant l'id numéro 51. (le code est adapté à ce modèle. Pour l'autre site internet mit dans le rapport en bibliographie, il faut utiliser l'autre fichier projet_CNN_FP.ipynb)
Dans le code qui suit, la matrice d'embeddings n'est pas utilisé car elle donne de moins bon résultats si elle est utilisé.
Néamoins le code est en commentaire s'il doit être utilisé. Il faut décommenter à deux endroits qui sont indiqués dans le code

In [ ]:
# On utilise uniquement le synopsis et le titre comme données d'entrée
X_train = film_df.titre + ' ' + film_df.synopsis
# Les noms des classes sont remplacées par leur identifiant (un entier positif)
y_train = film_df.genre.map(class_index)
keras_vectorizer = get_vectorizer(X_train)
voc = keras_vectorizer.get_vocabulary()
# on associe un identifiant unique à chaque item du vocabulaire
word_index = dict(zip(voc, range(len(voc))))
# Chargement des plongements du fichier model.txt
m_embeddings = load_embeddings('model.txt')

In [ ]:
def get_embedding_matrix(vocabulary, embeddings_index, embedding_dim = 100):
  num_tokens = len(vocabulary)
  hits = 0
  misses = 0
  # Préparation de la matrice
  # Les mots qui ne se trouvent pas dans les plongements pré-entraînés seront 
  # représentés par des vecteurs dont toutes les composantes sont égales à 0,
  # y compris la représentation utilisée pour compléter les documents courts et
  # celle utilisée pour les mots inconnus [UNK]
  embedding_matrix = np.zeros((num_tokens, embedding_dim))
  for word, i in word_index.items():
      embedding_vector = embeddings_index.get(word)
      if embedding_vector is None or embedding_vector.shape == (0,) :
        misses += 1     
      else:
        embedding_matrix[i] = embedding_vector[:100] #certain on 101 au lieu de 100
        hits += 1
  print(f'{hits} mots ont été trouvés dans les plongements pré-entraînés')
  print(f'{misses} sont absents')
  return embedding_matrix
# Construction de la matrice de plongements à partir du vocabulaire
m_embedding_matrix = get_embedding_matrix(voc, m_embeddings)

In [ ]:

# Pour utiliser la matrice d'embedding, il faut décommenter le code ici
def get_CNN_model(voc_size, embedding_matrix, embedding_dim=100):
  # Création du modèle
  int_sequences_input = keras.Input(shape=(None,), dtype="int64")
  embedding_layer = Embedding(voc_size, embedding_dim, trainable=True,
      #embeddings_initializer=keras.initializers.Constant(embedding_matrix), 
  )
  
  embedded_sequences = embedding_layer(int_sequences_input)
  x = layers.Conv1D(64, 5, activation="relu")(embedded_sequences)
  x = layers.MaxPooling1D(5)(x)
  x = layers.Conv1D(64, 5, activation="relu")(x)
  x = layers.GlobalMaxPooling1D()(x)
  x = layers.Dense(64, activation="relu")(x)
  x = layers.Dropout(0.5)(x)
  preds = layers.Dense(len(class_names), activation="softmax")(x)
  model = keras.Model(int_sequences_input, preds)
  return model

In [ ]:
# Fonction pour l'entraînement d'un modèle
def train_model(X, y, model_function, vectorizer,
                voc_size, embedding_matrix, embedding_dim=100, batch_size=128):
  # Listes utilisées pour sauvegarder les résultats obtenus à chaque pli
  acc_per_fold = []
  loss_per_fold = []
  histories = []
  folds = 5
  stratkfold = model_selection.StratifiedKFold(n_splits=folds, shuffle=True, 
                                              random_state=12)
  fold_no = 1
  # Pour utiliser la matrice d'embedding, il faut aussi décommenter le code ici
  for train, test in stratkfold.split(X, y):
    m_function = globals()[model_function]
    model = m_function(voc_size, 
                      #embedding_matrix, 
                       embedding_dim)

    print('------------------------------------------------------------------------')
    print(f'Entraînement pour le pli {fold_no} ...')
    fold_x_train = vectorizer(X.iloc[train].to_numpy()).numpy()
    fold_x_val = vectorizer(X.iloc[test].to_numpy()).numpy()
    fold_y_train = y.iloc[train].to_numpy()
    fold_y_val = y.iloc[test].to_numpy()

    # Compilation du modèle : permet de préciser la fonction de perte et l'optimiseur
    # loss=sparse_categorical_crossentropy : entropie croisée, dans le cas où les 
    #  classes cibles sont indiquées sous forme d'entiers. Il s'agira de minimiser
    #  la perte pendant l'apprentissage
    # optimizer=rmsprop : l'optimiseur détermine la manière doit les poids seront
    #  mis à jour pendant l'apprentissage
    model.compile(
      loss="sparse_categorical_crossentropy", optimizer="rmsprop", metrics=["acc"]
    )
    # Entraînement sur 10 époque
    history = model.fit(fold_x_train, fold_y_train, batch_size=batch_size, 
                        epochs=10, validation_data=(fold_x_val, fold_y_val))
    histories.append(history)
    # Evaluation sur les données de validation
    scores = model.evaluate(fold_x_val, fold_y_val, verbose=0)
    print(f'Scores pour le pli {fold_no}: {model.metrics_names[0]} = {scores[0]:.2f};',
          f'{model.metrics_names[1]} = {scores[1]*100:.2f}%')
    acc_per_fold.append(scores[1] * 100)
    loss_per_fold.append(scores[0])
    fold_no = fold_no + 1

  # Affichage des scores moyens par pli
  print('---------------------------------------------------------------------')
  print('Scores par pli')
  for i in range(0, len(acc_per_fold)):
    print('---------------------------------------------------------------------')
    print(f'> Pli {i+1} - Loss: {loss_per_fold[i]:.2f}',
          f'- Accuracy: {acc_per_fold[i]:.2f}%')
  print('---------------------------------------------------------------------')
  print('Scores moyens pour tous les plis :')
  print(f'> Accuracy: {np.mean(acc_per_fold):.2f}',
        f'(+- {np.std(acc_per_fold):.2f})')
  print(f'> Loss: {np.mean(loss_per_fold):.2f}')
  print('---------------------------------------------------------------------')
  return histories

In [ ]:
def plot_results(histories):
  accuracy_data = []
  loss_data = []
  for i, h in enumerate(histories):
    acc = h.history['acc']
    val_acc = h.history['val_acc']
    loss = h.history['loss']
    val_loss = h.history['val_loss']
    for j in range(len(acc)):
      accuracy_data.append([i+1, j+1, acc[j], 'Entraînement'])
      accuracy_data.append([i+1, j+1, val_acc[j], 'Validation'])
      loss_data.append([i+1, j+1, loss[j], 'Entraînement'])
      loss_data.append([i+1, j+1, val_loss[j], 'Validation'])

  acc_df = pd.DataFrame(accuracy_data, 
                        columns=['Pli', 'Epoch', 'Accuracy', 'Données'])
  sns.relplot(data=acc_df, x='Epoch', y='Accuracy', hue='Pli', style='Données',
              kind='line')
    
  loss_df = pd.DataFrame(loss_data, columns=['Pli', 'Epoch', 'Perte', 'Données'])
  sns.relplot(data=loss_df, x='Epoch', y='Perte', hue='Pli', style='Données',
              kind='line')

In [ ]:
# Entraînement du modèle et récupération des résultats
CNN_histories = train_model(X_train, y_train, 'get_CNN_model',
                            keras_vectorizer, len(voc), m_embedding_matrix)

In [ ]:
plot_results(CNN_histories)